In [ ]:
# -*- coding: utf-8 -*-

In [ ]:
import numpy as np
import laspy

with laspy.open(r"E:\RTE_entrainement\IA_Train_Valid_Test\Dev_YSO_py\pred_25.laz") as fh:
    print('Points from Header:', fh.header.point_count)
    las = fh.read()
    print(las)
    print(las.xyz.shape)
    print('Points from data:', len(las.points))
    unique_classes,class_counts= np.unique(las.classification, return_counts=True)
    class_counts = dict(zip(unique_classes, class_counts))
    print(unique_classes)
    print(len(unique_classes))
    ground_pts = las.classification
   
    
    bins_g, counts_g = np.unique(las.return_number[ground_pts], return_counts=True)
    print('Ground Point Return Number distribution:')
    for r,c in zip(bins_g,counts_g):
        print('  {}:{}'.format(r,c))
        
    for clas in unique_classes:
        print(f'{clas},":", {las.xyz}')
        
    print(ground_pts)
    print(las.points)
  

# Main script for metrics

In [1]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import laspy
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,jaccard_score
import matplotlib.pyplot as plt 
from tqdm import tqdm

# Here, we're going to set, useful fonctions or potentialy useful 

def fIoU (tp,fp,fn) :
    return tp / (tp + fp + fn)
#intersection over union


# Foncton to calculate the precision of our model prediction
def Metricsecall (tp,fn) :
    
    #nombre : nombre de points classifiés dans une certaine classe par le nuage de référence
    if (tp+fn) == 0 :
        return 0
    else :
        return tp/(tp+fn)
    
def faccuracy(tp,fp,fn):
    return tp/(tp+fp+fn)
    
#nombre de points dans la même classification (les true points en cas de nuage de référence) / nombre de point de la classe 
#spécifique dans le nuage 1
#qualitativement c'est la proportion de points correctements classifiés par rapport aux points dans la classif
# recall == 1 signifie que l'algo n'omet pas de points


# Foncton to calculate the precision of our model prediction

def fprecision (tp,fp) :
    if (tp+fp) == 0 :
        return 0
    else :
        return tp/(tp+fp)
# precision == 1 signifie que que l'algo ne prend pas de points en trop

# here, is for a specific comparison between precision and the recall 
def ff1score (precision,rappel) :
    if (precision+rappel) < 10**-5 :
        return 0
    return 2*(precision*rappel)/(precision+rappel)


# =======This fonction retutrns general inforrmation about the file=====================#
#========possible to edit the script in oder to define your own output willing==========#

def infos_class(file_path):
    with laspy.open(file_path) as fh:
        print('Points Metricsom Header:', fh.header.point_count)
        las = fh.read()
        print(las.xyz.shape)
        print('Points Metricsom data:', len(las.points))
        unique_classes,class_counts= np.unique(las.classification, return_counts=True)
        class_counts = dict(zip(unique_classes, class_counts))
        print(class_counts)
        #print(las.classification)
        #  Display number of return
        
        """ground_pts= las.classification
        bins_g, counts_g = np.unique(las.return_number[ground_pts], return_counts=True)
        print('Ground Point Return Number distribution:')
        for r,c in zip(bins_g,counts_g):
            print('    {}:{}'.format(r,c))"""


# =============This part is only for reading las files====================================#
# ==============It returns something like this: with the number of points=================#
#=<LasData(1.4, point fmt: <PointFormat(6, 8 bytes of extra dims)>, 1258912 points, 1 vlrs)>=#

def read_file(file_path):
    with laspy.open(file_path) as fh:
        data=fh.read()
    return data
#
# ===================reading data and  extracting specific information,===================# 
# ====================like the occurences of classes in the file==========================#

def read_file_class(file_path):
    with laspy.open(file_path) as fh:
        data=fh.read()
    return data.classification
#
#====== Textract data label (different existing classes inside las files)=================#

def data_label(las):
    unique_classes,class_counts= np.unique(las.classification, return_counts=True)
    class_counts = dict(zip(unique_classes, class_counts))    
    return unique_classes

# this fonction is for Extracting Data 3D cordonnates Metricsom laZ files
def label_xyz(las):
    label_xyz = las.xyz
    return label_xyz


# ==============this fonction allow to count number of points in a class===============#

def count_points_per_class(file_path):
    class_counts = {}
    with laspy.open(file_path) as fh:
        las = fh.read()
        unique_classes, class_counts = np.unique(las.classification, return_counts=True)
        class_counts = dict(zip(unique_classes, class_counts))
    return class_counts


#
# ==========Extract each label of class in its occurence in the global point cloud======#

def num_points(classe):
    num_point_clas=[]
    for el in tqdm(range(len(classe))):
        num_point_clas.append((classe[el]))
    return num_point_clas

# ====================Compute matrix of confusion======================================#


def compute_confusion_matrix(ground_truth, predictions):
    """
    Calcule la matrice de confusion pour les prédictions et les classes réelles.
    Args:
        predictions (numpy.ndarray): Tableau des prédictions du modèle.
        ground_truth (numpy.ndarray): Tableau des classes réelles.
    Returns:
        pandas.DataFrame: Matrice de confusion avec les index et
        colonnes représentant les classes.
    """
    # Identifier les classes uniques dans ground_truth et predictions
    unique_classes = np.unique(np.concatenate((predictions, ground_truth)))
    
    # Créer une matrice de confusion basée uniquement sur les classes uniques
    confusion_matrix = np.zeros((len(unique_classes), len(unique_classes)))
    print("len ground_truth",len(ground_truth))
    print("len pred",len(predictions))
    # Remplir la matrice de confusion uniquement pour les classes existantes
    for i in tqdm(range(len(predictions))):
        pred_class = predictions[i]
        true_class = ground_truth[i]
        
        # Vérifier si la prédiction a une classe correspondante dans les données réelles
        if true_class in unique_classes:
            pred_index = np.where(unique_classes == pred_class)[0][0]
            true_index = np.where(unique_classes == true_class)[0][0]
            confusion_matrix[pred_index, true_index] += 1

    # Créer des titres de classe basés sur les classes uniques
    class_titles = [f"Classe {c}" for c in unique_classes]

    # Créer le DataFrame de la matrice de confusion avec les titres de classe
    conf_matrix_df = pd.DataFrame(confusion_matrix, index=class_titles, columns=class_titles)
    
    return conf_matrix_df




class Metrics:
    
    #
    # ====================Initializing my class Metrics================================================#
    
    def __init__(self, true_labels, pred_labels) :
        self.true_labels=true_labels
        self.pred_labels=pred_labels
    
    #
    # ====================Display metrics simply as a dictionnary======================================#    
    def compute_metrics(self):
        """ return metrics by class"""
        IoU=jaccard_score(self.true_labels, self.pred_labels, average=None)
        precision = precision_score(self.true_labels, self.pred_labels, average=None,zero_division='warn')
        recall = recall_score(self.true_labels, self.pred_labels, average=None,zero_division='warn')
        f1 = f1_score(self.true_labels, self.pred_labels, average=None)
        accuracy = accuracy_score(self.true_labels, self.pred_labels)
        
        metrics_dict = {
        'IoU': IoU,
        'Recall': recall,
        'Precision': precision,
        'F1_score': f1,
        'Accuracy': accuracy
    }
        return metrics_dict
    # 
    
    # ====================Display metrics bay class as a frame  without weighting =====================#
    
    def metrics_frame(self,results):
        unique_classes = np.unique(np.concatenate((self.true_labels, self.pred_labels)))
        # Créez des titres de classe basés sur les classes uniques
        class_titles = [f"Classe {c}" for c in unique_classes]
        nam_metrics = ['IoU', 'Recall', 'Precision', 'F1_score', 'Accuracy']
        Metrics = np.zeros((5, len(results['IoU'])))
        
        for i, metric_name in enumerate(nam_metrics):
            Metrics[i] = results[metric_name]    
        #  
        # Créez le DataMetricsame de la matrice de confusion avec les titres de classe
        Metrics_df = pd.DataFrame(Metrics, index=nam_metrics, columns=class_titles)
        Metrics_df.index.name = 'Métriques'
        Metrics_df.columns.name = 'Valeurs des métriques'
        return Metrics_df
        
     # ====================Display metrics as a frame by weghting each class===========================#   
    def weighted_metrics(self):
        
        """Calculate metrics for each label, and find their average weighted 
        by support (the number of true instances for each label)"""
        
        IoU_w=jaccard_score(self.true_labels, self.pred_labels, average="weighted")
        precision_w = precision_score(self.true_labels, self.pred_labels, average='weighted',zero_division='warn')
        recall_w = recall_score(self.true_labels, self.pred_labels, average='weighted',zero_division='warn')
        f1_w = f1_score(self.true_labels, self.pred_labels, average='weighted')
        accuracy = accuracy_score(self.true_labels, self.pred_labels)
        
        nam_metrics = ['IoU', 'Recall','precision','Accuracy', 'F1_score']
        weigh_metrics=[IoU_w,recall_w,precision_w,f1_w,accuracy]
        weigh_metrics_df=pd.DataFrame({'Valeurs': weigh_metrics}, index=nam_metrics)
        weigh_metrics_df.index.name = 'Métriques'
        
        return weigh_metrics_df
        
        
     # ====================Display metrics as a graphics curves======================================#   
        
    def graphmetrics(self,metrics_data):        
        data=np.arange(1, len(metrics_data['IoU'])+1)
        plt.figure(figsize=(15,7))
        # Tracé des métriques
        for metric, values in metrics_data.items():
            if metric != 'Accuracy':             
                plt.plot(data, values, label=metric)
        
        # Ajout de légendes et de titres
        plt.xlabel('Classes')
        plt.ylabel('Valeur')
        plt.title('Métriques par classe')
        plt.legend()
        plt.grid(True)

            # Affichage du graphique
        plt.tight_layout()
        plt.show()
            
    # ====================Display metrics as subplots graphics on grids===============================#
    
    def n_graphmetrics(self, metrics_data):        
        data = np.arange(1, len(metrics_data['IoU']) + 1)
        metrics_to_plot = [metric for metric in metrics_data if metric != 'Accuracy']
        num_metrics = len(metrics_to_plot)
        
        fig, axes = plt.subplots(num_metrics, 1, figsize=(15, 7*num_metrics))
        
        for i, (metric, values) in enumerate(metrics_data.items()):
            if metric != 'Accuracy':
                ax = axes[i] if num_metrics > 1 else axes  # Utiliser le même axe s'il n'y a qu'une seule métrique
                ax.plot(data, values, label=metric)
                ax.set_xlabel('Classes')
                ax.set_ylabel('Valeur')
                ax.set_title(f'Métrique: {metric}')
                ax.legend()
                ax.grid(True)  # Ajout de la grille
                
        plt.tight_layout()
        plt.show()
        
        # Filter metrics according to the models config
        
    def filt_metrics(data_frame, model):
       model2 = {"0":1, "1":2, "2":5, "3":6, "4":196, "5":211}
       class_names = {"Défaut":1,"Sol": 2, "Végégtation":5,"Bâtiments":6, "Structures": 196, "Câbles": 211 }
             

                    
        
        

In [2]:
model_dict = {"1":"default","2":"ground","5": "vegetation", "6":"building","7": "wall", "196": "structure","211":"cable"}

conv_class = {"0":"default", "1": "ground", "2": "vegetation", "3": "building", "4": "structure", "5": "cable"}

conv_dict2 = {"1": 0, "2": 1, "3": 0, "4": 2, "5": 2, "6": 3, "7": 6,  "12": 0, "13": 0, "15": 0, "50": 0,
     "51": 0, "52": 0, "53": 0,"54": 0, "55": 0,  "64": 0,"72": 0,"101": 0,"102": 4,"103": 4, "104": 4,
      "105": 4,"106": 0,"107": 0,"108": 0,"109": 0, "110": 0,"111": 0, "120": 0, "121": 0,"150": 0,
      "151": 0,"152": 5,"153": 5,"154": 5, "155": 5,"160": 0,"161": 0,"185": 0,"196": 4,"197": 4,
      "200": 5,"201": 5,"202": 5,"211": 5,"212": 5, "213": 5,"221": 5,"222": 5, "223": 5
     }

model = {k:int(l) for k, v in conv_class.items() for l,m in model_dict.items() if v == m}
#model2 = {"0":1, "1":2, "2":5, "3":6, "4":196, "5":211}
direct_conv2={k:j for k, v in conv_dict2.items() for  i, j in model.items()  if int(i) == v }
print(model)

{'0': 1, '1': 2, '2': 5, '3': 6, '4': 196, '5': 211}


In [3]:
f = read_file_class(r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte2 correctement labelise/99385bdd1b4245d6813754f478bd5f2c/preds/000006.laz")
print(np.unique(f))

In [ ]:
infos_class(r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/db3ea63f541a4d9fbe68bd96f98a86ae/preds/000006.laz")
read_file_class(r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/db3ea63f541a4d9fbe68bd96f98a86ae/preds/000006.laz")

## Test part for understanding the issue of mappind the input data with json file config

In [ ]:

data = read_file_class(r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/Controle lignes elec-1_YAM.laz")
print(np.unique(data))
print(type(data))

conv_dict1 = {"1": 0,"2": 1,"3": 0,"4": 2,"5": 2, "6": 3,"7": 8,"12": 0,"13": 0,"15": 0,"50": 4,"51": 4,
             "52": 4, "53": 4,"54": 4, "55": 4, "64": 4,"72": 7, "101": 5,"102": 5,"103": 5, "104": 5,
          "105": 5,"106": 5,"107": 5, "108": 5,"109": 5,"110": 5,"111": 5,"120": 5,"121": 5,"150": 6, "151": 6,
      "152": 6,"53": 6, "154": 6,"155": 6, "160": 6, "161": 6, "185": 6, "196": 5,
      "197": 5,"200": 6,"201": 6, "202": 6,"211": 6,"212": 6,"213": 6, "221": 6, "222": 6,"223": 6
  }
print(conv_dict1)

model1 = {"0":1, "1":2, "2":5, "3":6, "4":59, "5":196, "6": 211, "7":72}

#

direct_conv1={k:j for k, v in conv_dict1.items() for  i, j in model1.items()  if int(i) == v }

print(direct_conv1)
direct1 = [direct_conv1.get(str(k)) for k in data]

conv_dict2 = {"1": 0, "2": 1, "3": 0, "4": 2, "5": 2, "6": 3, "7": 6,  "12": 0, "13": 0, "15": 0, "50": 0,
     "51": 0, "52": 0, "53": 0,"54": 0, "55": 0,  "64": 0,"72": 0,"101": 0,"102": 4,"103": 4, "104": 4,
      "105": 4,"106": 0,"107": 0,"108": 0,"109": 0, "110": 0,"111": 0, "120": 0, "121": 0,"150": 0,
      "151": 0,"152": 5,"153": 5,"154": 5, "155": 5,"160": 0,"161": 0,"185": 0,"196": 4,"197": 4,
      "200": 5,"201": 5,"202": 5,"211": 5,"212": 5, "213": 5,"221": 5,"222": 5, "223": 5
     }
model2 = {"0":1, "1":2, "2":5, "3":6, "4":196, "5":211}
direct_conv2={k:j for k, v in conv_dict2.items() for  i, j in model2.items()  if int(i) == v }




direct2 = [direct_conv2.get(str(k)) for k in data]
print("")


# Supposons que direct soit votre tableau NumPy
# Nettoyer le tableau pour supprimer les valeurs None
cleaned_direct = [x for x in direct1 if x is not None]

# Utiliser np.unique() sur le tableau nettoyé
unique_values = np.unique(cleaned_direct)

# Afficher les valeurs uniques
print("unique values")
print(type(unique_values))
print(unique_values)

print(type(np.array(direct1)))
# Convertir les valeurs de data en utilisant le dictionnaire de conversion

# Convertir les valeurs de data en utilisant le dictionnaire de conversion
converted_data1 = [conv_dict1.get(str(value), value) for value in data]
print(np.unique(converted_data1))

converted_data2 = [conv_dict2.get(str(value), value) for value in data]
print(np.unique(converted_data2))




conv_class = {
        "0": "default",
        "1": "ground",
        "2": "vegetation",
        "3": "building",
        "4": "structure",
        "5": "cable"
    }

[  6  50  60  64  67  68  79 102 103 104 105 106 107 108 109 110 111 150
 151 152 153 154 155 160 161 196 197 200 201 211 212 213 221 222 223]
<class 'numpy.ndarray'>
{'1': 0, '2': 1, '3': 0, '4': 2, '5': 2, '6': 3, '7': 8, '12': 0, '13': 0, '15': 0, '50': 4, '51': 4, '52': 4, '53': 6, '54': 4, '55': 4, '64': 4, '72': 7, '101': 5, '102': 5, '103': 5, '104': 5, '105': 5, '106': 5, '107': 5, '108': 5, '109': 5, '110': 5, '111': 5, '120': 5, '121': 5, '150': 6, '151': 6, '152': 6, '154': 6, '155': 6, '160': 6, '161': 6, '185': 6, '196': 5, '197': 5, '200': 6, '201': 6, '202': 6, '211': 6, '212': 6, '213': 6, '221': 6, '222': 6, '223': 6}
{'1': 1, '2': 2, '3': 1, '4': 5, '5': 5, '6': 6, '12': 1, '13': 1, '15': 1, '50': 59, '51': 59, '52': 59, '53': 211, '54': 59, '55': 59, '64': 59, '72': 72, '101': 196, '102': 196, '103': 196, '104': 196, '105': 196, '106': 196, '107': 196, '108': 196, '109': 196, '110': 196, '111': 196, '120': 196, '121': 196, '150': 211, '151': 211, '152': 211, '154': 2

## Model3

In [3]:
conv_dict2 = {"1": 0, "2": 1, "3": 0, "4": 2, "5": 2, "6": 3, "7": 6,  "12": 0, "13": 0, "15": 0, "50": 0,
     "51": 0, "52": 0, "53": 0,"54": 0, "55": 0,  "64": 0,"72": 0,"101": 0,"102": 4,"103": 4, "104": 4,
      "105": 4,"106": 0,"107": 0,"108": 0,"109": 0, "110": 0,"111": 0, "120": 0, "121": 0,"150": 0,
      "151": 0,"152": 5,"153": 5,"154": 5, "155": 5,"160": 0,"161": 0,"185": 0,"196": 4,"197": 4,
      "200": 5,"201": 5,"202": 5,"211": 5,"212": 5, "213": 5,"221": 5,"222": 5, "223": 5
     }
model2 = {"0":1, "1":2, "2":5, "3":6, "4":196, "5":211}
direct_conv2={k:j for k, v in conv_dict2.items() for  i, j in model2.items()  if int(i) == v }



# Exemple d'utilisation
rte2_pred=read_file_class(r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte3/9a275f9320714938a25097c942b35477/preds/000006.laz")
rte2_truth=read_file_class(r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz")
pred_class=num_points(rte2_pred)
gtruth_class=num_points(rte2_truth)

#num_ground = [x for x in num_ground if x is not None]
#num_ground=[direct_conv2.get(str(k)) for k in gtruth_class]
gtruth_cleanead = [direct_conv2.get(str(k)) if direct_conv2.get(str(k)) is not None else k for k in gtruth_class]

rte2_predictions = np.array(pred_class)
print("unique classe predictions")
print(len(rte2_predictions))
print(np.unique(rte2_predictions))
print("")

rte2_ground_truth =np.array(gtruth_cleanead)
print("unique classe ground truth")
print("")
print("len gtruth_class",len(gtruth_class))
print("gtruth unique classes",np.unique(gtruth_class))
print("")
print("len gtruth_class_merged", len(rte2_ground_truth))
print("unique gtruth lasses merged classe", np.unique(rte2_ground_truth))


#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
conf_matrix = compute_confusion_matrix(rte2_ground_truth,rte2_predictions)

print("Matrice de confusion :/n", conf_matrix)

## Instancing
rte2_metrics=Metrics(rte2_ground_truth,rte2_predictions)
## calling a methods for computing metrics
rte2=rte2_metrics.compute_metrics()
rte2_metrics.weighted_metrics()


100%|██████████| 113550333/113550333 [00:12<00:00, 9164199.40it/s] 


unique classe predictions
113550333
[  1   2   5   6 196 211]

unique classe ground truth

len gtruth_class 113550333
gtruth unique classes [  2   3   4   5   6  14  50  51  56  70  71  72  79 106 107 111 150 151
 161 196 197 211 212 213]

len gtruth_class_merged 113550333
unique gtruth lasses merged classe [  1   2   5   6  14  56  70  71  79 196 211]
len ground_truth 113550333
len pred 113550333


100%|██████████| 113550333/113550333 [06:51<00:00, 275946.45it/s]


Matrice de confusion :/n               Classe 1   Classe 2    Classe 5  Classe 6  Classe 14  Classe 56  \
Classe 1    33115384.0  1846776.0    458013.0  450914.0      248.0    16148.0   
Classe 2    12549409.0  1433781.0      3885.0   30323.0        0.0      333.0   
Classe 5     8162659.0   267088.0  53900527.0  207145.0      695.0     2778.0   
Classe 6      480644.0    40658.0     35627.0  374585.0      392.0        0.0   
Classe 14          0.0        0.0         0.0       0.0        0.0        0.0   
Classe 56          0.0        0.0         0.0       0.0        0.0        0.0   
Classe 70          0.0        0.0         0.0       0.0        0.0        0.0   
Classe 71          0.0        0.0         0.0       0.0        0.0        0.0   
Classe 79          0.0        0.0         0.0       0.0        0.0        0.0   
Classe 196      4890.0        1.0     46356.0      79.0      671.0       15.0   
Classe 211     10397.0      377.0       968.0       0.0      743.0        0.0   

  

c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Valeurs
Métriques,
IoU,0.692586
Recall,0.782988
precision,0.862058
Accuracy,0.802300
F1_score,0.782988


In [4]:
rte2_metrics.metrics_frame(rte2)

Valeurs des métriques,Classe 1,Classe 2,Classe 5,Classe 6,Classe 14,Classe 56,Classe 70,Classe 71,Classe 79,Classe 196,Classe 211
Métriques,,,,,,,,,,,
IoU,0.579857,0.088654,0.854286,0.231105,0.000000,0.000000,0.000000,0.000000,0.000000,0.107562,0.857804
Recall,0.609597,0.399529,0.989993,0.352370,0.000000,0.000000,0.000000,0.000000,0.000000,0.513910,0.996602
Precision,0.922394,0.102282,0.861727,0.401751,0.000000,0.000000,0.000000,0.000000,0.000000,0.119745,0.860320
F1_score,0.734063,0.162869,0.921417,0.375443,0.000000,0.000000,0.000000,0.000000,0.000000,0.194232,0.923460
Accuracy,0.782988,0.782988,0.782988,0.782988,0.782988,0.782988,0.782988,0.782988,0.782988,0.782988,0.782988


In [ ]:
rte2_metrics.weighted_metrics()

In [ ]:
infos_class(r"D:/YSO/Nouveau dossier/Controle lignes elec-1_YAM.laz")
print("")
infos_class(r"D:/YSO/Controle lignes elec-1_YAM.laz")

## Part of Correcting the input issue of converrted data

## Rte2: second predictions

##### Model2 : file :000006.laz

In [ ]:
# Exemple d'utilisation
rte2_pred=read_file_class(r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte2 correctement labelise/99385bdd1b4245d6813754f478bd5f2c/preds/000006.laz")
rte2_truth=read_file_class(r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz")
pred_class=num_points(rte2_pred)
gtruth_class=num_points(rte2_truth)

num_ground = [direct_conv2.get(str(k)) if direct_conv2.get(str(k)) is not None else k for k in gtruth_class]
#num_ground=[direct_conv2.get(str(k)) for k in gtruth_class]
gtruth_cleanead = [x for x in num_ground if x is not None]

rte2_predictions = np.array(pred_class)
print("unique classe predictions")
print(len(rte2_predictions))
print(np.unique(rte2_predictions))
print("")

rte2_ground_truth =np.array(gtruth_cleanead)
print("unique classe ground truth")
print("")
print("len gtruth_class",len(gtruth_class))
print("gtruth unique classes",np.unique(gtruth_class))
print("")
print("len gtruth_class_merged", len(rte2_ground_truth))
print("unique gtruth lasses merged classe", np.unique(rte2_ground_truth))


#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
conf_matrix = compute_confusion_matrix(rte2_ground_truth,rte2_predictions)

print("Matrice de confusion :/n", conf_matrix)

## Instancing
rte2_metrics=Metrics(rte2_ground_truth,rte2_predictions)
## calling a methods for computing metrics
rte2=rte2_metrics.compute_metrics()
rte2_metrics.weighted_metrics()
## for displaying as frame




## jasonload

In [ ]:
rte2_metrics.metrics_frame(rte2)

In [ ]:
rte2_metrics.graphmetrics(rte2)


In [ ]:
rte2_metrics.n_graphmetrics(rte2)

## Model1: file 000006.laz

In [ ]:
# Exemple d'utilisation
rte1_pred=read_file_class(r"E:/S_Données_pour apprentissage/data_output inference/Donnees  labelises en sortie d'inference/rte1 correctement labelise/db3ea63f541a4d9fbe68bd96f98a86ae/preds/000006.laz")
rte1_truth=read_file_class(r"E:/S_Données_pour apprentissage/Données pour prédiction sans bruit/000006.laz")
pred_class=num_points(rte1_pred)
gtruth_class=num_points(rte1_truth)

num_ground = [direct_conv1.get(str(k)) if direct_conv1.get(str(k)) is not None else k for k in gtruth_class]
#num_ground=[direct_conv1.get(str(k)) for k in gtruth_class]
gtruth_cleanead = [x for x in num_ground if x is not None]

rte1_predictions = np.array(pred_class)
print("unique classe predictions")
print(len(rte1_predictions))
print(np.unique(rte1_predictions))
print("")

rte1_ground_truth =np.array(gtruth_cleanead)
print("unique classe ground truth")
print("")
print("len gtruth_class",len(gtruth_class))
print("gtruth unique classes",np.unique(gtruth_class))
print("")
print("len gtruth_class_merged", len(rte1_ground_truth))
print("unique gtruth lasses merged classe", np.unique(rte1_ground_truth))


#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
conf_matrix = compute_confusion_matrix(rte1_ground_truth,rte1_predictions)

print("Matrice de confusion :/n", conf_matrix)

## Instancing
rte1_metrics=Metrics(rte1_ground_truth,rte1_predictions)
## calling a methods for computing metrics
rte1=rte1_metrics.compute_metrics()

## for displaying as frame
rte1_metrics.graphmetrics(rte1)
rte1_metrics.n_graphmetrics(rte1)


rte1_metrics.weighted_metrics()


In [ ]:
rte1_metrics.metrics_frame(rte1)

### Model2 : file Controle lignes elec-1_YAM.laz

In [ ]:
# Exemple d'utilisation
rte2_pred=read_file_class(r"D:/YSO/Nouveau dossier/Controle lignes elec-1_YAM.laz")
rte2_truth=read_file_class(r"D:/YSO/Controle lignes elec-1_YAM.laz")
pred_class=num_points(rte2_pred)
gtruth_class=num_points(rte2_truth)

num_ground = [direct_conv2.get(str(k)) if direct_conv2.get(str(k)) is not None else k for k in gtruth_class]
#num_ground=[direct_conv2.get(str(k)) for k in gtruth_class]
gtruth_cleanead = [x for x in num_ground if x is not None]

rte2_predictions = np.array(pred_class)
print("unique classe predictions")
print(len(rte2_predictions))
print(np.unique(rte2_predictions))
print("")
rte2_ground_truth =np.array(gtruth_cleanead)
print("unique classe ground truth")
print("")
print("gtruth_class",len(gtruth_class))
print(len(rte2_ground_truth))
print(np.unique(rte2_ground_truth))

"""print("type and unique pred_calss")
print(type(pred_class))
print(np.unique(pred_class))
print("rte2_pred")
print(np.unique(rte2_predictions))
print(type(rte2_predictions))

print("")

print("cleanead")
print(type(gtruth_cleanead))
print("rte2_gtruth")
print(type(rte2_ground_truth))



print("gtruth_class")
print(np.unique(gtruth_class))
print(type(gtruth_class))"""






#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
conf_matrix = compute_confusion_matrix(rte2_ground_truth,rte2_predictions)

print("Matrice de confusion :\n", conf_matrix)

## Instancing
rte2_metrics=Metrics(rte2_ground_truth,rte2_predictions)
## calling a methods for computing metrics
rte2=rte2_metrics.compute_metrics()
rte2_metrics.weighted_metrics()
## for displaying as frame




rte2_metrics.graphmetrics(rte2)
rte2_metrics.n_graphmetrics(rte2)

## jasonload

In [ ]:
rte2_metrics.metrics_frame(rte2)

In [ ]:
rte2_metrics.weighted_metrics()

In [ ]:
#count_points_per_class(r"D:/YSO/Controle lignes elec-1_YAM.laz")

# Rte1 first predictions

## Model1 file Controle lignes elec-1_YAM.laz

In [ ]:

# Exemple d'utilisation
rte1_pred=read_file_class(r"D:/YSO/rte1/Controle lignes elec-1_YAM.laz")
rte1_truth=read_file_class(r"D:/YSO/Controle lignes elec-1_YAM.laz")
pred_class=num_points(rte1_pred)
gtruth_class=num_points(rte1_truth)

num_ground = [direct_conv2.get(str(k)) if direct_conv2.get(str(k)) is not None else k for k in gtruth_class]
#num_ground=[direct_conv2.get(str(k)) for k in gtruth_class]
gtruth_cleanead = [x for x in num_ground if x is not None]

rte1_predictions = np.array(pred_class)
print("unique classe predictions")
print(len(rte1_predictions))
print(np.unique(rte1_predictions))
print("")
rte1_ground_truth =np.array(gtruth_cleanead)
print("unique classe ground truth")
print("")
print("gtruth_class",len(gtruth_class))
print(len(rte1_ground_truth))
print(np.unique(rte1_ground_truth))

"""print("type and unique pred_calss")
print(type(pred_class))
print(np.unique(pred_class))
print("rte1_pred")
print(np.unique(rte1_predictions))
print(type(rte1_predictions))

print("")

print("cleanead")
print(type(gtruth_cleanead))
print("rte1_gtruth")
print(type(rte1_ground_truth))



print("gtruth_class")
print(np.unique(gtruth_class))
print(type(gtruth_class))"""






#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
conf_matrix = compute_confusion_matrix(rte1_ground_truth,rte1_predictions)

print("Matrice de confusion :\n", conf_matrix)

## Instancing
rte1_metrics=Metrics(rte1_ground_truth,rte1_predictions)
## calling a methods for computing metrics
rte1=rte1_metrics.compute_metrics()
rte1_metrics.weighted_metrics()
## for displaying as frame




rte1_metrics.graphmetrics(rte1)
rte1_metrics.n_graphmetrics(rte1)

## jasonload

In [ ]:
rte1_metrics.metrics_frame(rte1)

In [ ]:
rte1_metrics.weighted_metrics()

# Part without mapping the input file with json config file

In [ ]:
# Exemple d'utilisation
rte2_pred=read_file_class(r"E:\S_Données_pour apprentissage\data_post_Pred\Nouveau dossier\99385bdd1b4245d6813754f478bd5f2c\preds\000006.laz")
rte2_truth=read_file_class(r"E:\S_Données_pour apprentissage\Données pour prédiction sans bruit\000006.laz")
num_classe1=num_points(rte2_pred)
num_classe2=num_points(rte2_truth)
rte2_predictions = np.array(num_classe1)
rte2_ground_truth = np.array(num_classe2)
#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
conf_matrix = compute_confusion_matrix(rte2_ground_truth,rte2_predictions)

print("Matrice de confusion :\n", conf_matrix)

## Instancing
rte2_metrics=Metrics(rte2_ground_truth,rte2_predictions)
## calling a methods for computing metrics
rte2=rte2_metrics.compute_metrics()
rte2_metrics.weighted_metrics()
## for displaying as frame
rte2_metrics.metrics_frame(rte2)

#metrics.graphmetrics(rte2)


#compute_metrics_per_class(rte2_ground_truth,rte2_predictions)
#calculate_metrics(conf_matrix.values)


#calculate_metrics(conf_matrix.values)

# Écrire le Datrte2rame dans un fichier CSV



In [ ]:
rte2_metrics.weighted_metrics()

In [ ]:
rte2_metrics.graphmetrics(rte2)
rte2_metrics.n_graphmetrics(rte2)

In [ ]:
# Exemple d'utilisation
c1=read_file_class(r"E:\RTE_entrainement\IA_Train_Valid_Test\Dev_YSO_py\pred_25.laz")
c2=read_file_class(r"E:\RTE_entrainement\IA_Train_Valid_Test\Dev_YSO_py\before_25.laz")
num_classe1=num_points(c1)
num_classe2=num_points(c2)
predictions = np.array(num_classe1)
ground_truth = np.array(num_classe2)
#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
conf_matrix = compute_confusion_matrix(ground_truth,predictions)

print("Matrice de confusion :\n", conf_matrix)

## Instancing
metrics=Metrics(ground_truth,predictions)
## calling a methods for computing metrics
af=metrics.compute_metrics()
metrics.weighted_metrics()
## for displaying as frame
metrics.metrics_frame(af)

#metrics.graphmetrics(af)


#compute_metrics_per_class(ground_truth,predictions)
#calculate_metrics(conf_matrix.values)


#calculate_metrics(conf_matrix.values)

# Écrire le DataFrame dans un fichier CSV



In [ ]:
metrics.weighted_metrics()

In [ ]:

# Exemple d'utilisation
c1=read_file_class(r"E:\RTE_entrainement\IA_Train_Valid_Test\Dev_YSO_py\pred_25.laz")
c2=read_file_class(r"E:\RTE_entrainement\IA_Train_Valid_Test\Dev_YSO_py\before_25.laz")
num_classe1=num_points(c1)
num_classe2=num_points(c2)
predictions = np.array(num_classe1)
ground_truth = np.array(num_classe2)
#class_titles = ["Classe 0", "Classe 1","classes2","classe3"]
#conf_matrix = compute_confusion_matrix(ground_truth,predictions)

print("Matrice de confusion :\n", conf_matrix)

## Instancing
metrics=Metrics(ground_truth,predictions)
## calling a methods for computing metrics
af=metrics.compute_metrics()
metrics.graphmetrics(af)
metrics.n_graphmetrics(af)